In [ ]:
import tensorflow as tf 
import keras 
import numpy as np
import pandas as pd
from keras.layers import Dense, Dropout, Embedding
from keras.models import Sequential, Model 
from keras.optimizers import SGD 
from keras.utils import np_utils

from tensorflow.keras.datasets import cifar10, mnist

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
import keras.backend as K

data = "farhan is here bacha , be carefull of, him."

words = data.split()
words = [x.lower().replace(",","").replace(".","") for x in words]
vocab = set(words)
vocab_len = len(vocab)
word_to_idx = {j:i for i,j in enumerate(words)}
idx_to_word = {i:j for i,j in enumerate(words)} 

c=2
dim=100
X_train = []
Y_train = []

for index, i in enumerate(words):
    
    #edge case
    if index <= 1 or index >= len(words)-2:
        continue
        
    # sliding context window 
    start = index - c
    end = index + c + 1
    # 2 words Left and 2 words Right
    context = words[start:end]
    context.remove(i)
    target = i
    
    # data
    X_train.append([word_to_idx[w] for w in context])
    Y_train.append([word_to_idx[target]])
X_train = np.array(X_train)
Y_train = np.array(Y_train)
Y_train = np_utils.to_categorical(Y_train, vocab_len)

cbow = Sequential() 
cbow.add(Embedding(input_dim=vocab_len, output_dim=dim, input_length= c*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(vocab_len, activation='softmax'))

cbow.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=["accuracy"])
cbow.fit(X_train, Y_train, epochs=5)

pred = cbow.predict(X_train)

pred = pred.argmax(axis=1)

i = 0
for context in X_train:
    print("...............................")
    print("===========Context :===========")
    for w in context:
        print(idx_to_word[w])
    print("=========Predicted Target :====")
    print(idx_to_word[pred[i]])
    i+=1